In [1]:
from fastbook import *


# 协同过滤 collaborative filtering

协同过滤是一种在推荐系统中常用的方法，通过分析用户和项目之间的关系，来发现其他用户对于这种项目的潜在的兴趣，比如说在B站里面，你看了很多二次元动画片，b站就会给你推荐很多其他也喜欢看二次元动画片的人也喜欢看的东西。

这次用的数据集是一个叫MovieLens的数据集，主要包含了电影的一些数据。

In [2]:
from fastai.collab import *
from fastai.tabular.all import *
path = untar_data(URLs.ML_100k)

In [3]:
ratings = pd.read_csv(path/'u.data', delimiter='\t', header=None,
                      names=['user','movie','rating','timestamp'])
ratings.head()

,user,movie,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


这个数据集中有用户的编号，电影的编号，用户对于电影的评分，还有个时间戳

接下来，我们需要将电影的元素用一个向量来表示，比如第一个参数表示多大程度上是科幻片，第二个是动作电影的程度，第三个是这个电影是不是老电影，用一个（-1,1）的域来表示。比如这个《星战最后的绝地武士》这个电影（名字我猜的，我没看过），就是典型的科幻动作电影，但是不是老电影

In [5]:
last_skywalker = np.array([0.98,0.9,-0.9])

相同的我们也可以将用户的属性来表示出来，比如这个用户比较喜欢动作和科幻电影，但是不喜欢老电影

In [6]:
user1 = np.array([0.9,0.8,-0.6])

我们把这两个向量相乘,这个结果就是用户对于这个电影的喜欢程度

PS:在pytorch和numpy中，点乘是*，叉乘是@

In [7]:
(user1*last_skywalker).sum()

2.1420000000000003

换个例子，这个卡萨布兰卡绝对不是一个科幻电影，动作成分也基本没有，但是是一个实打实的老电影，这样的电影碰上user1会如何呢？

In [8]:
casablanca = np.array([-0.99,-0.3,0.8])

In [9]:
(user1*casablanca).sum()

-1.611

结果上来讲是一个代表不喜欢的负值。

OK,将这个特例拓展到整个样本空间，这样的话我们就能得到一个所有用户对于所有电影的喜好矩阵了。